In [1]:
from __future__ import print_function
import os
import time

import numpy as np
import zipfile
import tarfile
from six.moves.urllib.request import urlretrieve
import shutil 
import random

import string
import tensorflow as tf

# Dirs - must be absolute paths!
LOG_DIR = '/tmp/tf/ptb_char_rnn/B100S10_H32_dynamic/'
# Local dir where PTB files will be stored.
PTB_DIR = '/home/tkornuta/data/ptb/'

# Filenames.
TRAIN = "ptb.train.txt"
VALID = "ptb.valid.txt"
TEST = "ptb.test.txt"

# Length of sequence (=  number of units of controller (recurrent layer))
SEQ_LENGTH = 10

# Size of batch
BATCH_SIZE = 100

# Length of a vector - a single hidden state
HIDDEN_SIZE = 32

# Size of the input vector - 59 = [A-Z] + [a-z] + ' '
INPUT_SIZE = 59

#### MANN-related parameters.
# Size of the local memory of each cell.
#NUMBER_OF_SLOTS = 20
# Length of a single vector - a single memory slot.
#ADDRESS_SIZE = NUMBER_OF_SLOTS
#CONTENT_SIZE = 8
#SLOT_SIZE = ADDRESS_SIZE + CONTENT_SIZE

# Eps for normalization in visualization
EPS = 1e-10

### Check/maybe download PTB.

In [2]:
def maybe_download_ptb(path, 
                       filename='simple-examples.tgz', 
                       url='http://www.fit.vutbr.cz/~imikolov/rnnlm/', 
                       expected_bytes =34869662):
  # Eventually create the PTB dir.
  if not tf.gfile.Exists(path):
    tf.gfile.MakeDirs(path)
  """Download a file if not present, and make sure it's the right size."""
  _filename = path+filename
  if not os.path.exists(_filename):
    print('Downloading %s...' % filename)
    _filename, _ = urlretrieve(url+filename, _filename)
  statinfo = os.stat(_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', (_filename), '(', statinfo.st_size, ')')
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + _filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download_ptb(PTB_DIR)

Found and verified /home/tkornuta/data/ptb/simple-examples.tgz ( 34869662 )


### Extract dataset-related files from the PTB archive.

In [3]:
def extract_ptb(path, filename='simple-examples.tgz', files=["ptb.train.txt", "ptb.valid.txt", "ptb.test.txt", 
                                       "ptb.char.train.txt", "ptb.char.valid.txt", "ptb.char.test.txt"]):
    """Extracts files from PTB archive."""
    # Extract
    tar = tarfile.open(path+filename)
    tar.extractall(path)
    tar.close()
    # Copy files
    for file in files:
        shutil.copyfile(PTB_DIR+"simple-examples/data/"+file, PTB_DIR+file)
    # Delete directory
    shutil.rmtree(PTB_DIR+"simple-examples/")        

extract_ptb(PTB_DIR)

### Load train, valid and test texts.

In [4]:
def read_data(filename, path):
    with open(path+filename, 'r') as myfile:
        data=myfile.read()# .replace('\n', '')
        return data

train_text = read_data(TRAIN, PTB_DIR)
train_size=len(train_text)
print(train_size, train_text[:100])

valid_text = read_data(VALID, PTB_DIR)
valid_size=len(valid_text)
print(valid_size, valid_text[:64])

test_text = read_data(TEST, PTB_DIR)
test_size=len(test_text)
print(test_size, test_text[:64])

5101618  aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote
399782  consumers may want to move their telephones a little closer to 
449945  no it was n't black monday 
 but while the new york stock excha


### Utility functions to map characters to vocabulary IDs and back.

In [5]:
first_letter = ord(string.ascii_uppercase[0]) # ascii_uppercase before lowercase! 
print("vocabulary size = ", INPUT_SIZE)
print(first_letter)

def char2id(char):
  """ Converts char to id (int) with one-hot encoding handling of unexpected characters"""
  if char in string.ascii_letters:# or char in string.punctuation or char in string.digits:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    # print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  """ Converts single id (int) to character"""
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

#print(len(string.punctuation))
#for i in string.ascii_letters:
#    print (i, char2id(i))


print(char2id('a'), char2id('A'), char2id('z'), char2id('Z'), char2id(' '), char2id('ï'))
print(id2char(char2id('a')), id2char(char2id('A')))
#print(id2char(65), id2char(33), id2char(90), id2char(58), id2char(0))
#bankno
sample = np.zeros(shape=(1, INPUT_SIZE), dtype=np.float)
sample[0, char2id(' ')] = 1.0
print(sample)

vocabulary size =  59
65
33 1 58 26 0 0
a A
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]


### Helper class for batch generation

In [6]:
class BatchGenerator(object):
  def __init__(self, text, batch_size, seq_length, vocab_size):
    """
    Initializes the batch generator object. Stores the variables and first "letter batch".
    text is text to be processed
    batch_size is size of batch (number of samples)
    seq_length represents the length of sequence
    vocab_size is number of words in vocabulary (assumes one-hot encoding)
    """
    # Store input parameters.
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._seq_length = seq_length
    self._vocab_size = vocab_size
    # Divide text into segments depending on number of batches, each segment determines a cursor position for a batch.
    segment = self._text_size // batch_size
    # Set initial cursor position.
    self._cursor = [ offset * segment for offset in range(batch_size)]
    # Store first "letter batch".
    self._last_letter_batch = self._next_letter_batch()
  
  def _next_letter_batch(self):
    """
    Returns a batch containing of encoded single letters depending on the current batch 
    cursor positions in the data.
    Returned "letter batch" is of size batch_size x vocab_size
    """
    letter_batch = np.zeros(shape=(self._batch_size, self._vocab_size), dtype=np.float)
    # Iterate through "samples"
    for b in range(self._batch_size):
      # Set 1 in position pointed out by one-hot char encoding.
      letter_batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return letter_batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    # First add last letter from previous batch (the "additional one").
    batches = [self._last_letter_batch]
    for step in range(self._seq_length):
      batches.append(self._next_letter_batch())
    # Store last "letter batch" for next batch.
    self._last_letter_batch = batches[-1]
    return batches


In [7]:
# Create objects for training, validation and testing batch generation.
train_batches = BatchGenerator(train_text, BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)

# For validation  - process the whole text as one big batch.
VALID_BATCH_SIZE = int(np.floor(valid_size/SEQ_LENGTH))
valid_batches = BatchGenerator(valid_text, VALID_BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)
# Get a single batch! 
valid_batch = valid_batches.next()

# For texting  - process the whole text as one big batch.
TEST_BATCH_SIZE = int(np.floor(test_size/SEQ_LENGTH))
test_batches = BatchGenerator(test_text, TEST_BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)
# Get a single batch! 
test_batch = test_batches.next()


### Helper functions - used during graph definition

In [8]:
# Function adding visualization to a given "matrix" (keys, memory etc.) with additional normalization
def visualize_hot_cold_normalized(matrix_,
                    axis_of_reduction_,
                    name_):
    with tf.name_scope(name_+"_vis_hc_norm"):
        # Create hot-cold visualization (red=positive/blue=negative)
        zeros = tf.zeros_like(matrix_) 
        
        # Get negative values only.
        neg = tf.less(matrix_, zeros)
        blue = tf.multiply(tf.cast(neg, tf.float32), matrix_)
        min_blue = tf.reduce_min(matrix_, axis=axis_of_reduction_, keep_dims=True) + EPS
        norm_blue = 255.0 * blue/min_blue
        
        # Get positive values only.
        pos = tf.greater(matrix_, zeros)
        red = tf.multiply(tf.cast(pos, tf.float32), matrix_)
        max_red = tf.reduce_max(matrix_, axis=axis_of_reduction_, keep_dims=True) + EPS
        norm_red = 255.0 * red/max_red
        
        # Stack them into three channel image with hot-cold values.
        rgb = tf.stack([norm_red, zeros, norm_blue], axis=2)

        #print("name_=",name_)
        #print("np.int32(zeros.shape[1]=", np.int32(zeros.shape[1]))
        # TODO: find fix and get rid of the batch_size(!)
        rgb_reshaped = tf.reshape(rgb, [1, -1,  np.int32(zeros.shape[1]), 3])

        # Visualize read weights as image.
        rgb_reshaped_summary = tf.summary.image(name_+"_visv", rgb_reshaped)

In [9]:
# Function adding visualization to a given "matrix" (keys, memory etc.)
def visualize_hot_cold(matrix_,
                    name_):
    with tf.name_scope(name_+"_vis_hc"):
        # Create hot-cold visualization (red=positive/blue=negative)
        zeros = tf.zeros_like(matrix_) 
        
        # Get negative values only.
        neg = tf.less(matrix_, zeros)
        blue = 255.0 * tf.multiply(tf.cast(neg, tf.float32), matrix_)
        
        # Get positive values only.
        pos = tf.greater(matrix_, zeros)
        red = 255.0 * tf.multiply(tf.cast(pos, tf.float32), matrix_)
        
        # Stack them into three channel image with hot-cold values.
        rgb = tf.stack([red, zeros, blue], axis=2)

        #print("name_=",name_)
        #print("np.int32(zeros.shape[1]=", np.int32(zeros.shape[1]))
        # TODO: find fix and get rid of the batch_size(!)
        rgb_reshaped = tf.reshape(rgb, [1, -1,  np.int32(zeros.shape[1]), 3])

        # Visualize read weights as image.
        rgb_reshaped_summary = tf.summary.image(name_+"_vis", rgb_reshaped)


In [10]:
# Function "fixing" tensordot lack of dimensions.
def tensordot_fix(matrix_a_, matrix_b_, axes_, output_shape_):
    tensor_without_dims = tf.tensordot(matrix_a_, matrix_b_, axes=axes_)
    # TF bugfix - tensordot not returning proper shapes when only partial shapes are known. :]
    # https://github.com/tensorflow/tensorflow/issues/6682
    tensor = tf.reshape(tensor_without_dims, output_shape_)
    return tensor 
    

###  Definition of tensor graph

In [27]:
def build_my_graph():
    # 1. Placeholders for inputs.
    with tf.name_scope("Input_data"):
        # Define input data buffers.
        data_buffers = list()
        for _ in range(SEQ_LENGTH + 1):
            # Collect placeholders for inputs/labels: Batch x Vocab size.
            data_buffers.append(tf.placeholder(tf.float32, shape=[None, INPUT_SIZE], name="data_buffers"))

        # Sequence of batches.
        input_seq_batch = data_buffers[:SEQ_LENGTH]

        # Labels are pointing to the same placeholders!
        # Labels are inputs shifted by one time step.
        labels_seq_batch = data_buffers[1:]  
        # Concatenate targets into 2D tensor.
        target_batch = tf.concat(labels_seq_batch, 0)

        # Add histograms to TensorBoard.
        input_seq_batch_hist = tf.summary.histogram("input_seq_batch", input_seq_batch)

    # 2. Unrolled controller ops.
    #with tf.name_scope("Controller"):
    # Placeholder for "initial" state [BATCH_SIZE X SLOT_SIZE].
    init_state = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_state")

    cell = tf.contrib.rnn.BasicRNNCell(HIDDEN_SIZE)
    rnn_outputs, final_state = tf.contrib.rnn.static_rnn(cell, input_seq_batch, initial_state=init_state)


    # Add histograms to TensorBoard.
    rnn_outputs_hist = tf.summary.histogram("rnn_outputs", rnn_outputs)

    # 3. Output ops.
    #logits and predictions
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [HIDDEN_SIZE, INPUT_SIZE])
        b = tf.get_variable('b', [INPUT_SIZE], initializer=tf.constant_initializer(0.0))
    logits_batch = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]
    prediction_batch = [tf.nn.softmax(logits_batch) for logit in logits_batch]


    # 4. Loss ops.
    with tf.name_scope("Loss"):
        # Loss function(s) - one for every output generated by every LSTM cell.
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            labels=target_batch, logits=logits_batch))
        # Add loss summary.
        loss_summary = tf.summary.scalar("loss", loss)

    # 5. Training ops.  
    with tf.name_scope("Optimization"):
        # Learning rate decay.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(0.1, global_step, 5000, 0.9, staircase=True)
        # Optimizer.
        optimizer = tf.train.AdamOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(loss)
        gradients, v = zip(*grads_and_vars)
        # Gradient clipping.
        gradients, _ = tf.clip_by_global_norm(gradients, 0.1)
        optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

        #for i, (grad, var) in enumerate(grads_and_vars):
        #    if grad is not None:
        #        gradients[i] = (tf.clip_by_value(grad, -10, 10), var)
        #        tf.summary.histogram(var.name, var)
        #        tf.summary.histogram(var.name + '/grad', grad)


    # Merge all summaries.
    merged_summaries = tf.summary.merge_all()

print("Graph definition OK")

Graph definition OK


In [32]:
# Reset graph - just in case.
tf.reset_default_graph()

t = time.time()
build_my_graph()
print("It took", time.time() - t, "seconds to build the graph.")
print(input_seq_batch)

It took 0.42838025093078613 seconds to build the graph.


NameError: name 'input_seq_batch' is not defined

In [29]:
def create_feed_dict(set_type_):
    """Creates feed dictionaries and set initial values for placeholders for different sets"""
    feed_dict = dict()
    
    if set_type_=="train":
        # Get next batch and create a feed dict.
        next_batch = train_batches.next()
        # Feed batch to input buffers.
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = next_batch[i]

        # Reset "initial" RNN state.
        feed_dict[init_state] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
            
    elif set_type_=="valid":
        # Feed batch to input buffers.
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = valid_batch[i]

        # Reset "initial" RNN state.
        feed_dict[init_state] = np.zeros([VALID_BATCH_SIZE, HIDDEN_SIZE])

    else: # test
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = test_batch[i]
        
        # Reset "initial" RNN state.
        feed_dict[init_state] = np.zeros([TEST_BATCH_SIZE, HIDDEN_SIZE])
       
    return feed_dict

print("Feed_dict definition OK")

Feed_dict definition OK


### Session execution

In [30]:
# Eventually clear the log dir.
if tf.gfile.Exists(LOG_DIR):
  tf.gfile.DeleteRecursively(LOG_DIR)
# Create (new) log dir.
tf.gfile.MakeDirs(LOG_DIR)

print("Log dir CLEARED")

Log dir CLEARED


In [31]:

# Create session to execute graph.
sess=tf.InteractiveSession()

# Create summary writers, point them to LOG_DIR.
train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
valid_writer = tf.summary.FileWriter(LOG_DIR + '/valid')
test_writer = tf.summary.FileWriter(LOG_DIR + '/test')

# Initialize global variables.
tf.global_variables_initializer().run()
print('Variables initialized')

# Initialize sparse memory addressing
#sess.run(init_sparse_memory_addressing)
    
# Determine how long to perform the training and how often the test loss on validation batch will be computed. 
summary_frequency = 100
validation_frequency = 1000
#num_steps = 10
num_steps = train_size // (BATCH_SIZE*SEQ_LENGTH)
print("Number of iterations per epoch =", num_steps)
for step in range(num_steps):
    input_seq_batch_, summaries, _, loss_, lr_ = sess.run([
        input_seq_batch, merged_summaries, optimizer, loss, learning_rate],
        feed_dict=create_feed_dict("train"))
    # Every (100) steps collect statistics.
    if step % summary_frequency == 0:
        # Add summary.
        train_writer.add_summary(summaries, step*BATCH_SIZE*SEQ_LENGTH)
        train_writer.flush()
        print('Training set BPC at step %d: %0.5f learning rate: %f' % (step, loss_, lr_))

        #print("memory=\n", memory_)
        # Print loss from last batch.
    
    if step % validation_frequency == 0:
        # Validation set BPC.
        print('=' * 80)
        print("Calculating BPC on validation set")
        v_summaries, v_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("valid"))
        print("Validation set BPC: %.5f" % v_loss)
        valid_writer.add_summary(v_summaries, step*BATCH_SIZE*SEQ_LENGTH)
        valid_writer.flush()
    # End of statistics collection

#for gv in grads_and_vars:
#    #print(str(sess.run(gv[0])) + " - " + gv[1].name)        
#    print(" - " + gv[1].name)        

# Test set BPC.
print('=' * 80)
print("Calculating BPC on test set")
t_summary, t_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("test"))
print("Final test set BPC: %.5f" % t_loss)
test_writer.add_summary(t_summary, step*BATCH_SIZE*SEQ_LENGTH)
test_writer.flush()
    
# Close writers and session.
train_writer.close()
valid_writer.close()
test_writer.close()
sess.close() 

Variables initialized
Number of iterations per epoch = 5101


NameError: name 'input_seq_batch' is not defined